In [ ]:
!pip install recordlinkage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 926 kB 26.4 MB/s 
     |████████████████████████████████| 132 kB 50.4 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73976 sha256=6111840ea8c11a1b6cbd53f7807436b725a233bc0e2e76da29c194519a3b0d76
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


https://recordlinkage.readthedocs.io/en/latest/guides/link_two_dataframes.html

In [ ]:
import recordlinkage
from recordlinkage.datasets import load_febrl4

dfA, dfB = load_febrl4()

# Indexation step
indexer = recordlinkage.Index()
indexer.block("given_name")
candidate_links = indexer.index(dfA, dfB)

# Comparison step
compare_cl = recordlinkage.Compare()

compare_cl.exact("given_name", "given_name", label="given_name")
compare_cl.string("surname", "surname", method="jarowinkler", threshold=0.85, label="surname")
compare_cl.exact("date_of_birth", "date_of_birth", label="date_of_birth")
compare_cl.exact("suburb", "suburb", label="suburb")
compare_cl.exact("state", "state", label="state")
compare_cl.string("address_1", "address_1", threshold=0.85, label="address_1")

features = compare_cl.compute(candidate_links, dfA, dfB)

# Classification step
matches = features[features.sum(axis=1) > 3]
print(len(matches))
print(matches.head())

3241
                             given_name  surname  date_of_birth  suburb  \
rec_id_1     rec_id_2                                                     
rec-2371-org rec-2371-dup-0           1      1.0              1       1   
rec-3024-org rec-3024-dup-0           1      1.0              1       0   
rec-4652-org rec-4652-dup-0           1      1.0              1       0   
rec-4795-org rec-4795-dup-0           1      1.0              1       1   
rec-1016-org rec-1016-dup-0           1      1.0              1       1   

                             state  address_1  
rec_id_1     rec_id_2                          
rec-2371-org rec-2371-dup-0      1        1.0  
rec-3024-org rec-3024-dup-0      1        0.0  
rec-4652-org rec-4652-dup-0      1        1.0  
rec-4795-org rec-4795-dup-0      1        1.0  
rec-1016-org rec-1016-dup-0      0        1.0  


https://recordlinkage.readthedocs.io/en/latest/guides/data_deduplication.html

In [ ]:
import recordlinkage
from recordlinkage.datasets import load_febrl1

dfA = load_febrl1()

# Indexation step
indexer = recordlinkage.Index()
indexer.block(left_on="given_name")
candidate_links = indexer.index(dfA)

# Comparison step
compare_cl = recordlinkage.Compare()

compare_cl.exact("given_name", "given_name", label="given_name")
compare_cl.string("surname", "surname", method="jarowinkler", threshold=0.85, label="surname")
compare_cl.exact("date_of_birth", "date_of_birth", label="date_of_birth")
compare_cl.exact("suburb", "suburb", label="suburb")
compare_cl.exact("state", "state", label="state")
compare_cl.string("address_1", "address_1", threshold=0.85, label="address_1")

features = compare_cl.compute(candidate_links, dfA)

# Classification step
matches = features[features.sum(axis=1) > 3]
print(len(matches))
print(matches.head())

317
                             given_name  surname  date_of_birth  suburb  \
rec_id_1      rec_id_2                                                    
rec-122-dup-0 rec-122-org             1      1.0              1       1   
rec-183-org   rec-183-dup-0           1      1.0              1       1   
rec-248-dup-0 rec-248-org             1      1.0              1       1   
rec-373-dup-0 rec-373-org             1      1.0              1       1   
rec-10-org    rec-10-dup-0            1      1.0              1       1   

                             state  address_1  
rec_id_1      rec_id_2                         
rec-122-dup-0 rec-122-org        1        1.0  
rec-183-org   rec-183-dup-0      1        1.0  
rec-248-dup-0 rec-248-org        1        1.0  
rec-373-dup-0 rec-373-org        1        1.0  
rec-10-org    rec-10-dup-0       1        1.0  


In [ ]:
import pandas
from recordlinkage.preprocessing import clean

names = ['Mary-ann',
        'Bob :)',
        'Angel',
        'Bob (alias Billy)',
        None]
print(names)
s = pandas.Series(names)
print(clean(s).values.tolist())

['Mary-ann', 'Bob :)', 'Angel', 'Bob (alias Billy)', None]
['mary ann', 'bob', 'angel', 'bob', None]


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import re


url = 'https://plo.vn/xu-ly-can-bo-sai-pham-khong-vung-cam-du-chuc-vu-cao-den-dau-post683629.html'

r = requests.get(url)

soup = bs(r.text, "html.parser")
content = soup.find_all("div", class_="article__body")
content = bs(str(content[0]),'html.parser')
content = soup.find_all("p", style="text-align: justify;")


def clean_text(text):
  return re.sub('<.*?>', '', str(text))

content = [clean_text(line) for line in content]
print(len(content))
for line in content:
  print(line)

20
 Cơ quan CSĐT Bộ Công an ngày 7-6 đã ra quyết định khởi tố bị can, bắt tạm giam ông Chu Ngọc Anh - cựu chủ tịch UBND TP Hà Nội, cựu Bộ trưởng KH&amp;CN; ông Nguyễn Thanh Long, cựu bộ trưởng Bộ Y tế và ông Phạm Công Tạc, cựu thứ trưởng Bộ KH&amp;CN để điều tra sai phạm liên quan vụ kit test Việt Á.
Trao đổi với Pháp luật TP.HCM, đại biểu Quốc hội (ĐBQH) Phạm Văn Hòa, đoàn Đồng Tháp, Ủy viên Ủy ban Pháp luật của QH đánh giá diễn biến tố tụng trên cho thấy quan điểm “không có vùng cấm” trong công tác phòng, chống tham nhũng của Đảng và Nhà nước.
. Phóng viên: Ông đánh giá như thế nào về việc Cơ quan CSĐT Bộ Công an cùng lúc khởi tố, bắt tạm giam ba cán bộ cấp cao, trong đó có tới hai người từng đứng đầu hai bộ ngành?
+ Ông Phạm Văn Hòa: Đây là một hành động hết sức cương quyết, thể hiện được quyết tâm của Đảng và Nhà nước trong công tác phòng chống tham nhũng, chống lãng phí. 
Thời gian qua, tình hình tham nhũng, tiêu cực, lãng phí trong một bộ phận cán bộ có chức có quyền của nước ta 

In [ ]:
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.5 MB 16.2 MB/s 
     |████████████████████████████████| 965 kB 39.7 MB/s 


In [ ]:
import unicodedata
import regex as re
from pyvi import ViTokenizer, ViPosTagger

bang_nguyen_am= [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]

bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']
nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

# Chuẩn hóa unicode 
# Có 2 loại unicode : unicode tổ hơp và unicode dựng sẵn, điêu này dẫn tới việc 2 từ giống nhau sẽ bị coi là khác nhau 
# Chuẩn hóa tất cả về 1 loại là unicode dựng sẵn
def chuan_hoa_unicode(text):
	text = unicodedata.normalize('NFC', text)
	return text

# Có 2 kiểu gõ dấu ở Tiếng Việt, ví dụ như là : òa hoặc oà (ta gọi lần lượt là chuẩn 1 và 2). Mặc dù kiểu gõ chữ sau ít 
#phổ biến hơn tuy nhiên vẫn cần phải chuẩn hóa tránh việc một số văn bản vẫn sử dụng kiểu gõ dấu thứ 2.
"""
	Hàm này xử lý chuẩn hóa từng từ một, sau khi chuẩn hóa từng từ thì ta sẽ đi chuân hóa từng câu sau 
	""" 
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
 
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
 
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)
 
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)
 
 
def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    # sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^/p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

# Tách từ tiếng việt, từ tiếng việt không giống như tiếng anh, tách từ tiếng anh ta chỉ cần tách bằng khoảng trắng
# Tuy nhiên từ tiếng Việt có cả từ đơn lẫn từ ghép nên tách từ tiêng Việt sẽ phúc tạp hơn 
# Project sử dung thu viện pyvi (xem mã nguồn tại : https://github.com/trungtv/pyvi) để phục vụ bài toán con tách từ Tiếng Việt 
def tach_tu_tieng_viet(text):
	text = ViTokenizer.tokenize(text)
	return text

# Đưa về chữ viết thường 
def chuyen_chu_thuong(text):
	return text.lower()

# Xóa đi các dấu cách thừa, các từ không cần thiết cho việc phân loại vẳn bản 
def chuan_hoa_cau(text):
	text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',text)
	text = re.sub(r'\s+', ' ', text).strip()
	return text

def tien_xu_li(text):
	text = chuan_hoa_unicode(text)
	text = chuan_hoa_dau_cau_tieng_viet(text)
	text = tach_tu_tieng_viet(text)
	text = chuyen_chu_thuong(text)
	text = chuan_hoa_cau(text)

	return text

In [ ]:
text = content[0]
print(text)
print(chuan_hoa_unicode(text))
print(chuan_hoa_dau_cau_tieng_viet(text))
print(chuan_hoa_cau(text))
print(tach_tu_tieng_viet(text))
print(tien_xu_li(text))

 Cơ quan CSĐT Bộ Công an ngày 7-6 đã ra quyết định khởi tố bị can, bắt tạm giam ông Chu Ngọc Anh - cựu chủ tịch UBND TP Hà Nội, cựu Bộ trưởng KH&amp;CN; ông Nguyễn Thanh Long, cựu bộ trưởng Bộ Y tế và ông Phạm Công Tạc, cựu thứ trưởng Bộ KH&amp;CN để điều tra sai phạm liên quan vụ kit test Việt Á.
 Cơ quan CSĐT Bộ Công an ngày 7-6 đã ra quyết định khởi tố bị can, bắt tạm giam ông Chu Ngọc Anh - cựu chủ tịch UBND TP Hà Nội, cựu Bộ trưởng KH&amp;CN; ông Nguyễn Thanh Long, cựu bộ trưởng Bộ Y tế và ông Phạm Công Tạc, cựu thứ trưởng Bộ KH&amp;CN để điều tra sai phạm liên quan vụ kit test Việt Á.
Cơ quan CSĐT Bộ Công an ngày 7-6 đã ra quyết định khởi tố bị can, bắt tạm giam ông Chu Ngọc Anh - cựu chủ tịch UBND TP Hà Nội, cựu Bộ trưởng KH&amp;CN; ông Nguyễn Thanh Long, cựu bộ trưởng Bộ Y tế và ông Phạm Công Tạc, cựu thứ trưởng Bộ KH&amp;CN để điều tra sai phạm liên quan vụ kit test Việt Á.
Cơ quan CSĐT Bộ Công an ngày 7 6 đã ra quyết định khởi tố bị can bắt tạm giam ông Chu Ngọc Anh cựu chủ t

In [ ]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
choices = ["Lê Thanh Tùng", "Le Thanh Tung", "le thanhtùng", "le Tah tùg", "le Thah tùgn", "Le Tung", "Nguyễn Thành Long", "Lê Mỹ Hạnh", "Lê Thanh Lân", "Lê Hoàng Tùng", "Lưu Hoàng Tùng"]

print(process.extract("Lê Thanh Tùng", choices,limit=10))

res = process.extractBests("Lê Thanh Tùng", choices,limit=10,score_cutoff=80)
print(res)

print(process.extractOne("Lê Thanh Tùng", choices))

print(fuzz.ratio("Lê","Lưu"))
print(fuzz.ratio("Thanh","Hoàng"))
print(fuzz.ratio("Tùng","Lân"))

# def compare_text(name,candidate):
#   text = 

[('Lê Thanh Tùng', 100), ('Le Thanh Tung', 92), ('le thanhtùng', 91), ('Lê Thanh Lân', 86), ('le Thah tùgn', 82), ('le Tah tùg', 80), ('Lê Hoàng Tùng', 76), ('Lưu Hoàng Tùng', 70), ('Nguyễn Thành Long', 59), ('Lê Mỹ Hạnh', 57)]
[('Lê Thanh Tùng', 100), ('Le Thanh Tung', 92), ('le thanhtùng', 91), ('Lê Thanh Lân', 86), ('le Thah tùgn', 82), ('le Tah tùg', 80)]
('Lê Thanh Tùng', 100)
40
20
29
